In [1]:
import pandas as pd
import pymc as pm
import numpy as np
import arviz as az

# Carregar os dados
data = pd.read_excel('ECQ_FEV25.xlsx')

# Verificar as colunas disponíveis
print("Colunas do dataset:", data.columns)

# Converter as variáveis categóricas para códigos numéricos
data['anf_idx'] = data['ANF'].astype('category').cat.codes
data['mun_idx'] = data['MUNICIPIO'].astype('category').cat.codes

# Criar um mapeamento que relacione cada município ao seu ANF
# Assumindo que cada município pertença a um único ANF
mun_to_anf = data.drop_duplicates('mun_idx').sort_values('mun_idx')['anf_idx'].values

# Arrays com os dados observados
n_tests = data['TESTES_ECQ'].values
n_success = data['TESTES_ECQ_OK'].values

# Número total de observações (sites) e de níveis hierárquicos
n_sites = data.shape[0]
n_anfs = data['anf_idx'].nunique()
n_mun = data['mun_idx'].nunique()

c:\Users\F8058552\AppData\Local\anaconda3\envs\clusters\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install gxx`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.


Colunas do dataset: Index(['ANF', 'MUNICIPIO', 'ENDERECO_ID', 'TESTES_ECQ', 'TESTES_ECQ_OK'], dtype='object')


# Treinamento
------------

In [ ]:

# Modelo vetorizado
with pm.Model() as modelo_vectorizado:
    # Dados observados via pm.Data para permitir atualizações sem recompilar o modelo
    n_tests_data = pm.Data("n_tests", n_tests)
    n_success_data = pm.Data("n_success", n_success)
    
    # Nível 3: Parâmetros para as ANFs
    mu_anf = pm.Beta("mu_anf", alpha=2, beta=2, shape=n_anfs)
    sigma_anf = pm.HalfNormal("sigma_anf", sigma=0.1, shape=n_anfs)
    
    # Nível 2: Parâmetros para os municípios
    # Aqui, definimos um phi individual para cada município para capturar a concentração
    phi_mun = pm.HalfNormal("phi_mun", sigma=10, shape=n_mun)
    
    # Parâmetros para os municípios, condicionado ao nível de ANF
    mu_mun = pm.Beta(
        "mu_mun",
        alpha=mu_anf[mun_to_anf] * phi_mun,
        beta=(1 - mu_anf[mun_to_anf]) * phi_mun,
        shape=n_mun
    )
    
    # Nível 1: Parâmetro para os sites (observações individuais)
    phi_site = pm.HalfNormal("phi_site", sigma=10)
    
    # Cada site é associado ao seu município via data["mun_idx"]
    theta_site = pm.Beta(
        "theta_site",
        alpha=mu_mun[data['mun_idx'].values] * phi_site,
        beta=(1 - mu_mun[data['mun_idx'].values]) * phi_site,
        shape=n_sites
    )
    
    # Likelihood: Modelo Binomial para cada observação
    obs = pm.Binomial(
        "obs",
        n=n_tests_data,
        p=theta_site,
        observed=n_success_data
    )
    
    # Amostragem
    trace = pm.sample(4000, tune=100, chains=4, target_accept=0.9)


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [mu_anf, sigma_anf, phi_mun, mu_mun, phi_site, theta_site]


Output()

In [3]:
# Converter o trace para InferenceData
idata = az.InferenceData(
    posterior=trace.posterior,
    sample_stats=trace.sample_stats,
    observed_data=trace.observed_data
)

# Salvar em NetCDF (garanta que o caminho está correto)
idata.to_netcdf("trace_ECQ_.nc")

NameError: name 'trace' is not defined

In [9]:
# leia o modelo em trace_ECC.nc
idata = az.from_netcdf("trace_ECQ_.nc")

In [10]:
# 2. De cada ANF, quais os piores municípios?
# Os municípios estão modelados via o parâmetro mu_mun, que depende do ANF
# a que pertencem. Para cada ANF, você pode:
# Obter o resumo dos parâmetros dos municípios.
#
# Usar o mapeamento (vetor mun_to_anf) para agrupar os municípios por ANF.
# Dentro de cada grupo, identificar os municípios com menores médias.
# Obter o resumo do parâmetro mu_mun
summary_mun = az.summary(idata, var_names=["mu_mun"])
summary_mun = summary_mun.reset_index().rename(columns={'index': 'mun_idx'})

# Converter o mun_idx para inteiro e associar ao ANF correspondente
summary_mun['mun_idx'] = summary_mun['mun_idx'].str.extract('(\d+)').astype(int)
summary_mun['anf'] = [mun_to_anf[i] for i in summary_mun['mun_idx']]

# Agrupar por ANF e identificar o município com menor média
piores_municipios = summary_mun.groupby('anf').apply(lambda df: df.nsmallest(1, 'mean'))
print("Piores municípios por ANF:")
print(piores_municipios[['mun_idx', 'mean']])


Piores municípios por ANF:
          mun_idx   mean
anf                     
0   1200     1200  0.408
1   523       523  0.057
2   632       632  0.064
3   1207     1207  0.037
4   752       752  0.061
5   318       318  0.097
6   610       610  0.081
7   842       842  0.063
8   1340     1340  0.073
9   1215     1215  0.122
10  493       493  0.078
11  1038     1038  0.059
12  434       434  0.035
13  288       288  0.077
14  1333     1333  0.045


C:\Users\F8058552\AppData\Local\Temp\ipykernel_18312\2376533117.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  piores_municipios = summary_mun.groupby('anf').apply(lambda df: df.nsmallest(1, 'mean'))


In [11]:
import arviz as az
import pandas as pd
import numpy as np

# Supondo que 'data' é o dataframe original e 'mun_to_anf' é o array que mapeia cada município ao seu ANF.
# Também assumindo que as colunas originais 'ANF', 'MUNICIPIO' e 'ENDERECO_ID' existem no dataframe.

# Crie os mapeamentos dos índices para os nomes originais
anf_categories = data['ANF'].astype('category').cat.categories
anf_mapping = dict(enumerate(anf_categories))  # chave: código numérico; valor: nome do ANF

mun_categories = data['MUNICIPIO'].astype('category').cat.categories
mun_mapping = dict(enumerate(mun_categories))  # chave: código numérico; valor: nome do município

# --- 1. Resumo dos parâmetros dos ANFs (mu_anf) ---
summary_anf = az.summary(idata, var_names=["mu_anf"], hdi_prob=0.95).reset_index()
# O índice vem como "mu_anf[0]", "mu_anf[1]", etc. Extraímos o número
summary_anf["ANF_idx"] = summary_anf["index"].str.extract(r'\[(\d+)\]').astype(int)
# Mapeia para o nome real
summary_anf["ANF"] = summary_anf["ANF_idx"].map(anf_mapping)
# Reorganiza as colunas
df_anf = summary_anf[["ANF_idx", "ANF", "mean", "hdi_2.5%", "hdi_97.5%"]]
print("Resumo por ANF:")
print(df_anf.head())

# --- 2. Resumo dos parâmetros dos Municípios (mu_mun) ---
summary_mun = az.summary(idata, var_names=["mu_mun"], hdi_prob=0.95).reset_index()
summary_mun["mun_idx"] = summary_mun["index"].str.extract(r'\[(\d+)\]').astype(int)
# Mapeia para o nome do município
summary_mun["MUNICIPIO"] = summary_mun["mun_idx"].map(mun_mapping)
# Adiciona o índice do ANF usando o vetor mun_to_anf
summary_mun["ANF_idx"] = summary_mun["mun_idx"].apply(lambda i: mun_to_anf[i])
summary_mun["ANF"] = summary_mun["ANF_idx"].map(anf_mapping)
# Organiza as colunas desejadas
df_mun = summary_mun[["mun_idx", "MUNICIPIO", "ANF_idx", "ANF", "mean", "hdi_2.5%", "hdi_97.5%"]]
print("\nResumo por Município:")
print(df_mun.head())

# --- 3. Resumo dos parâmetros dos Sites (theta_site) ---
summary_site = az.summary(idata, var_names=["theta_site"], hdi_prob=0.95).reset_index()
summary_site["site_idx"] = summary_site["index"].str.extract(r'\[(\d+)\]').astype(int)

# Crie um dataframe dos metadados dos sites a partir do original.
# Supondo que cada linha do 'data' corresponde a um site e que o índice do dataframe
# corresponde ao site_idx usado no modelo.
df_site_meta = data.reset_index().rename(columns={'index': 'site_idx'})
# Se já não existir, garanta que as colunas "mun_idx" e "ANF" estejam presentes.
# Se necessário, adicione-as:
if "mun_idx" not in df_site_meta.columns:
    df_site_meta["mun_idx"] = data['MUNICIPIO'].astype('category').cat.codes
if "ANF" not in df_site_meta.columns:
    df_site_meta["ANF"] = data['ANF']

# Acrescente o nome do município e o nome do ANF, usando os mapeamentos:
df_site_meta["MUNICIPIO"] = df_site_meta["mun_idx"].map(mun_mapping)
# Também é possível adicionar o código do ANF a partir do vetor mun_to_anf:
df_site_meta["ANF_idx"] = df_site_meta["mun_idx"].apply(lambda i: mun_to_anf[i])
df_site_meta["ANF"] = df_site_meta["ANF_idx"].map(anf_mapping)

# Agora, combine o resumo de theta_site com os metadados do site.
df_site = pd.merge(summary_site, df_site_meta[["site_idx", "ENDERECO_ID", "MUNICIPIO", "ANF"]],
                   on="site_idx", how="left")
# Organize as colunas relevantes
df_site = df_site[["site_idx", "ENDERECO_ID", "MUNICIPIO", "ANF", "mean", "hdi_2.5%", "hdi_97.5%"]]
print("\nResumo por Site:")


Resumo por ANF:
   ANF_idx  ANF   mean  hdi_2.5%  hdi_97.5%
0        0   71  0.597     0.506      0.693
1        1   73  0.416     0.355      0.481
2        2   74  0.361     0.285      0.440
3        3   75  0.406     0.361      0.454
4        4   77  0.358     0.300      0.416

Resumo por Município:
   mun_idx     MUNICIPIO  ANF_idx  ANF   mean  hdi_2.5%  hdi_97.5%
0        0       ABAIARA       13   88  0.134     0.000      0.382
1        1        ABAIRA        4   77  0.228     0.000      0.488
2        2         ABARE        3   75  0.416     0.116      0.689
3        3  ABREU E LIMA        6   81  0.654     0.502      0.814
4        4     ACAJUTIBA        3   75  0.271     0.041      0.495

Resumo por Site:


In [12]:
# Salve o dataframe com o resumo dos parâmetros dos municípios e dos sites em sheets diferentes
with pd.ExcelWriter("resumo_municipios_sites.xlsx") as writer:
    df_mun.to_excel(writer, sheet_name="Municipios", index=False)
    df_site.to_excel(writer, sheet_name="Sites", index=False)

In [ ]:
# Supondo que o município "João Pessoa" esteja em df_mun e df_site
mun_nome = "JOAO PESSOA"

# Média e HDI do município (usando df_mun)
mun_info = df_mun[df_mun["MUNICIPIO"] == mun_nome].iloc[0]
mun_media = mun_info["mean"]

# Filtrar os sites desse município
sites_joao = df_site[df_site["MUNICIPIO"] == mun_nome]

# Selecionar os sites cujo limite superior do HDI (por exemplo, "hdi_97.0%") esteja abaixo da média do município.
mun_media_percentual = round(mun_media * 100, 2)
print(f"Média de {mun_nome}: {mun_media_percentual}%")
sites_abaixo = sites_joao[sites_joao["hdi_97.5%"] < mun_media]

print("ENDERECO_IDs significativamente abaixo da média do município:")
sites_abaixo = sites_abaixo.sort_values("hdi_97.5%", ascending=True)


Média de QUEIMADAS: 54.7%
ENDERECO_IDs significativamente abaixo da média do município:


,ENDERECO_ID,mean,hdi_2.5%,hdi_97.5%
2685,PBQUS_0001,0.362,0.199,0.538


### 4. Se preciso elevar o ECQ do município em 2 pontos percentuais, quais ENDERECO_IDs preciso atualizar?
----
Suponha que a meta seja aumentar a média do município em 0,02 (2 pontos percentuais). O procedimento pode ser:

Definir o novo limiar desejado:
limite_novo = mun_media + 0.02


Selecionar os sites do município cujo limite superior do HDI esteja abaixo desse novo alvo, ou seja, aqueles que provavelmente não atingem o desempenho desejado.

In [ ]:
# Insira o nome do município alvo
MUNICIPIO_ALVO = "QUEIMADAS"
# Meta de aumento em 2 pontos percentuais
alvo = 0.8

# Calcule o impacto para cada site: quanto maior, maior a prioridade
df_site["impacto"] = df_site["TESTES_ECQ"] * (alvo - df_site["mean"])

# Ordene os sites pelo indicador de impacto (maior valor tem maior potencial de melhoria)
df_site_impacto = df_site.sort_values("impacto", ascending=False)
print("Sites com maior potencial de impacto:")
print(df_site_impacto[["ENDERECO_ID", "mean", "TESTES_ECQ", "impacto"]].head())


# Média e HDI do município (usando df_mun)
mun_info = df_mun[df_mun["MUNICIPIO"] == MUNICIPIO_ALVO].iloc[0]
mun_media = mun_info["mean"]


print(f"Novo alvo para {MUNICIPIO_ALVO}: {alvo}")

# Filtrar os sites do município alvo
sites_alvo = df_site[df_site["MUNICIPIO"] == MUNICIPIO_ALVO]

# Selecionar sites cujo limite superior do HDI esteja abaixo do alvo
sites_para_atuar = sites_alvo[sites_alvo["hdi_97.5%"] < alvo]

print("ENDERECO_IDs para intervenção (ECQ abaixo do novo alvo):")
sites_para_atuar[["ENDERECO_ID", "mean", "hdi_2.5%", "hdi_97.5%"]].sort_values("hdi_97.5%", ascending=True)

# Calcule o impacto para cada site: quanto maior, maior a prioridade
df_ECQ_TESTES = data[['ENDERECO_ID', 'TESTES_ECQ']]
# merge df_
df_site = pd.merge(df_site, df_ECQ_TESTES, on='ENDERECO_ID')



Novo alvo para QUEIMADAS: 0.8
ENDERECO_IDs para intervenção (ECQ abaixo do novo alvo):


In [8]:
import arviz as az
import pandas as pd
import numpy as np

# Carregue o modelo salvo
idata = az.from_netcdf("trace_ECQ_.nc")

# --- 1. Resumo dos parâmetros dos ANFs (mu_anf) ---
summary_anf = az.summary(idata, var_names=["mu_anf"], hdi_prob=0.95).reset_index()
# Extraia o índice numérico: "mu_anf[0]", "mu_anf[1]", etc.
summary_anf["ANF_idx"] = summary_anf["index"].str.extract(r'\[(\d+)\]').astype(int)
# Mapeie para o nome real do ANF usando o mapeamento de categorias
anf_categories = data['ANF'].astype('category').cat.categories
anf_mapping = dict(enumerate(anf_categories))
summary_anf["ANF"] = summary_anf["ANF_idx"].map(anf_mapping)
# Organize as colunas desejadas
df_anf = summary_anf[["ANF_idx", "ANF", "mean", "hdi_2.5%", "hdi_97.5%"]]
print("Resumo por ANF:")
print(df_anf.head())

# --- 2. Resumo dos parâmetros dos Municípios (mu_mun) ---
summary_mun = az.summary(idata, var_names=["mu_mun"], hdi_prob=0.95).reset_index()
summary_mun["mun_idx"] = summary_mun["index"].str.extract(r'\[(\d+)\]').astype(int)
# Mapeie para o nome do município usando os dados originais
mun_categories = data['MUNICIPIO'].astype('category').cat.categories
mun_mapping = dict(enumerate(mun_categories))
summary_mun["MUNICIPIO"] = summary_mun["mun_idx"].map(mun_mapping)
# Utilize o vetor mun_to_anf para associar o índice do ANF
summary_mun["ANF_idx"] = summary_mun["mun_idx"].apply(lambda i: mun_to_anf[i])
summary_mun["ANF"] = summary_mun["ANF_idx"].map(anf_mapping)
# Organize as colunas desejadas
df_mun = summary_mun[["mun_idx", "MUNICIPIO", "ANF_idx", "ANF", "mean", "hdi_2.5%", "hdi_97.5%"]]
print("\nResumo por Município:")
print(df_mun.head())

# --- 3. Resumo dos parâmetros dos Sites (theta_site) ---
summary_site = az.summary(idata, var_names=["theta_site"], hdi_prob=0.95).reset_index()
summary_site["site_idx"] = summary_site["index"].str.extract(r'\[(\d+)\]').astype(int)

# Crie um DataFrame de metadados dos sites a partir do DataFrame original "data"
df_site_meta = data.reset_index().rename(columns={'index': 'site_idx'})
# Caso as colunas "mun_idx" e "ANF" não existam, gere-as:
if "mun_idx" not in df_site_meta.columns:
    df_site_meta["mun_idx"] = data['MUNICIPIO'].astype('category').cat.codes
df_site_meta["MUNICIPIO"] = df_site_meta["mun_idx"].map(mun_mapping)
df_site_meta["ANF_idx"] = df_site_meta["mun_idx"].apply(lambda i: mun_to_anf[i])
df_site_meta["ANF"] = df_site_meta["ANF_idx"].map(anf_mapping)

# Combine o resumo de theta_site com os metadados
df_site = pd.merge(summary_site, df_site_meta[["site_idx", "ENDERECO_ID", "MUNICIPIO", "ANF"]], on="site_idx", how="left")
df_site = df_site[["site_idx", "ENDERECO_ID", "MUNICIPIO", "ANF", "mean", "hdi_2.5%", "hdi_97.5%"]]

# --- 4. Mesclar o número de testes por site ---
df_ECQ_TESTES = data[['ENDERECO_ID', 'TESTES_ECQ']]
df_site = pd.merge(df_site, df_ECQ_TESTES, on='ENDERECO_ID', how="left")

# --- 5. Cálculo do Impacto ---
# Defina o alvo de ECQ
alvo = 0.8
# Impacto = número de testes * (alvo - performance atual)
df_site["impacto"] = df_site["TESTES_ECQ"] * (alvo - df_site["mean"])

# --- 6. Salvar o resultado final ---
# O DataFrame final conterá: MUNICIPIO, ANF, ENDERECO_ID, mean, hdi_2.5%, hdi_97.5% e impacto
df_resultado = df_site[["MUNICIPIO", "ANF", "ENDERECO_ID", "mean", "hdi_2.5%", "hdi_97.5%", "impacto"]]

# Exiba os primeiros registros do resultado
print("\nResultado Final:")
print(df_resultado.head())

# Opcional: salvar em um arquivo Excel
df_resultado.to_excel("resultado_impacto_sites.xlsx", index=False)


Resumo por ANF:
   ANF_idx  ANF   mean  hdi_2.5%  hdi_97.5%
0        0   71  0.597     0.506      0.693
1        1   73  0.416     0.355      0.481
2        2   74  0.361     0.285      0.440
3        3   75  0.406     0.361      0.454
4        4   77  0.358     0.300      0.416

Resumo por Município:
   mun_idx     MUNICIPIO  ANF_idx  ANF   mean  hdi_2.5%  hdi_97.5%
0        0       ABAIARA       13   88  0.134     0.000      0.382
1        1        ABAIRA        4   77  0.228     0.000      0.488
2        2         ABARE        3   75  0.416     0.116      0.689
3        3  ABREU E LIMA        6   81  0.654     0.502      0.814
4        4     ACAJUTIBA        3   75  0.271     0.041      0.495

Resultado Final:
     MUNICIPIO  ANF ENDERECO_ID   mean  hdi_2.5%  hdi_97.5%  impacto
0       ANADIA   82  ALAAD_0001  0.691     0.545      0.838    3.379
1       ANADIA   82  ALAAD_0002  0.611     0.477      0.746    8.316
2  AGUA BRANCA   82  ALABN_0001  0.590     0.399      0.791    3.570
3

### 2. Indicador de Impacto para Intervenção

Se o objetivo é identificar onde os investimentos terão maior impacto para elevar o ECQ do município, você pode criar um indicador que combine o “déficit” de desempenho com o volume de testes. Por exemplo, se o alvo para o ECQ for \( T \) (por exemplo, 80% ou 0,80), defina para cada site:

\[
\text{Impacto}_i = \text{TESTES}_i \times (T - \text{mean}_i)
\]

Sites com maior valor desse indicador representam oportunidades onde há muito volume e, ao mesmo tempo, um déficit de desempenho relevante.


In [ ]:
df_sites_cidade

,site_idx,ENDERECO_ID,MUNICIPIO,ANF,mean,hdi_2.5%,hdi_97.5%,TESTES_ECQ,impacto
2685,2685,PBQUS_0001,QUEIMADAS,83,0.362,0.199,0.538,25,10.950
2686,2686,PBQUS_0002,QUEIMADAS,83,0.792,0.717,0.865,108,0.864


In [ ]:
#salve os dataframes ANF, MUNICIPIO e SITE em um arquivo excel unico, colocando sheets para cada um
# Save the dataframes to a single Excel file with separate sheets
with pd.ExcelWriter("output_data.xlsx") as writer:
    df_anf.to_excel(writer, sheet_name="ANF", index=False)
    df_mun.to_excel(writer, sheet_name="MUNICIPIO", index=False)
    df_site.to_excel(writer, sheet_name="SITE", index=False)

PermissionError: [Errno 13] Permission denied: 'output_data.xlsx'